In [2]:
################################### DATA WRAPPER ##########################################
from __future__ import division
import numpy as np
import os
import matplotlib.pyplot as plt


# rounds x up or down such that the expectation value of x is preserved
def rand_round(x):
	whole = int(x)
	r = np.random.random()
	if r < x-whole:
		whole += 1
	return whole
	
#gives a rotated and mirrored version of d
def augment_data(d,i):
	rot = i//2
	flip = i % 2
	if flip == 0:
		d = np.fliplr(d)
	return np.rot90(d,rot)
	
#primitive data wrapper to be extended
class DataWrapper:
	def __init__(self, stichSize):
		print('loading files...')
		self.stichSize = stichSize
		direct = 'no_cuts_16b/'
		files = os.listdir(direct)
		f0 = np.load(direct+files[0])
		im0 = f0['arr_0']
		im_shape = im0.shape
		lab0 = f0['arr_1']
		lab_shape = (lab0.shape[0]-stichSize+1,lab0.shape[1]-stichSize+1,lab0.shape[2])
		self.N_channels = im_shape[2]
		self.N_out = lab_shape[2]
		self.images = np.zeros((len(files),)+im_shape,dtype=np.float32)
		self.labels = np.zeros((len(files),)+lab_shape,dtype=np.float32)
		for i in range(len(files)):
			temp = np.load(direct+files[i])
			self.images[i,:,:,:] = temp['arr_0']
			self.labels[i,:,:,:] = temp['arr_1'][(stichSize)//2:-(stichSize-1)//2,(stichSize)//2:-(stichSize-1)//2,:]
			temp.close()
		print('... files loaded. Analyzing data...')
		maximu = np.zeros(self.N_channels)
		minimu = np.zeros(self.N_channels)
		for i in range(self.N_channels):
			maximu[i] = np.max(self.images[:,:,:,i])
			minimu[i] = np.min(self.images[:,:,:,i])
			self.images[:,:,:,i] -= minimu[i]
			self.images[:,:,:,i] /= (1.*maximu[i]-minimu[i])
		
		self.counter = np.zeros(self.N_out,dtype = np.int64)
		self.class_rate = np.zeros(self.N_out)
		self.testclass_rate = np.zeros(self.N_out)
		classes = []
		for i in range(self.N_out):
			classes += [np.array(np.where(self.labels[:,:,:,i]>0.1))]
		self.test_set_classes = []
		self.train_set_classes = []
		for i in range(self.N_out):
			classes[i] = classes[i][:,np.random.permutation(classes[i].shape[1])]
			self.test_set_classes += [classes[i][:,:min(300,classes[i].shape[1])]]
			self.train_set_classes += [classes[i][:,min(300,classes[i].shape[1]):]]
			self.class_rate[i] = self.train_set_classes[i].shape[1]
			self.testclass_rate[i] = self.test_set_classes[i].shape[1]
		#should be a parameter between 0 and 1. 1: uniform over the '10' classes, 0: same rate as in whole set
#		print(self.class_rate)
		self.uniformness = 0.98
	
	def make_stiches(self,vec):
		magicx = np.arange(self.stichSize)[np.newaxis,:,np.newaxis]
		magicy = np.arange(self.stichSize)[np.newaxis,np.newaxis,:]
		resultLab = self.labels[vec[0],vec[1],vec[2],:]
		resultIm = self.images[vec[0,:,np.newaxis, np.newaxis],vec[1,:,np.newaxis,np.newaxis]+magicx,vec[2,:,np.newaxis,np.newaxis]+magicy, :]
		rot = np.random.randint(4)
		flip = np.random.randint(2)
		if flip == 0:
			resultIm = np.flip(resultIm,1)
		resultIm = np.rot90(resultIm,rot,axes=(1,2))
		return resultIm, resultLab[:,np.newaxis,np.newaxis,:]
		
	def get_trainsamps_of_class(self, cl,amount):
		if amount > self.class_rate[cl]:
			print( "Error: not enough samples of class " + str(cl))
		if(self.counter[cl] + amount >= self.class_rate[cl]):
			self.counter[cl] = 0
			self.train_set_classes[cl] = self.train_set_classes[cl][:,np.random.permutation(self.train_set_classes[cl].shape[1])]
		result = self.train_set_classes[cl][:, self.counter[cl]:self.counter[cl]+amount]
		self.counter[cl] += amount
		return result
		
	def get_testsamps_of_class(self, cl,amount):
		if amount > self.testclass_rate[cl]:
			print( "Error: not enough samples of class " + str(cl))
		result = self.test_set_classes[cl][:, :amount]
#		print(amount, result.shape)
		return result
	
	def get_random_points(self,n):
		target_shape = self.labels.shape
		result = np.zeros((3,n))
		for i in range(3):
			result[i] = np.random.randint(target_shape[i],size=n)
		return result
		
	def getMiniBatch(self, miniBatchSize):
		class_get = np.zeros(self.N_out,dtype = np.int64)
		rates = (1./self.class_rate)**(1.-self.uniformness)
		rates *= self.uniformness/np.sum(rates)
		for i in range(self.N_out):
			class_get[i] = rand_round(rates[i]*miniBatchSize)
		s = np.sum(class_get)
		if s > miniBatchSize:
			class_get[np.argmax(class_get)] -= s-miniBatchSize
		resultvec = np.zeros((3,miniBatchSize), dtype=np.intp)
		got = 0
		for i in range(self.N_out):
			resultvec[:,got:got+class_get[i]] = self.get_trainsamps_of_class(i,class_get[i])
			got += class_get[i]
		resultvec[:,got:] = self.get_random_points(miniBatchSize-got)
		resultvec = resultvec[:,np.random.permutation(miniBatchSize)]
		return self.make_stiches(resultvec)

	def getTestData(self,size=100, uniformness = -1):
		if uniformness == -1:
				uniformness = self.uniformness
		class_get = np.zeros(self.N_out,dtype = np.int64)
		rates = (1./self.testclass_rate)**(1.-uniformness)
		rates *= uniformness/np.sum(rates)
		for i in range(self.N_out):
			class_get[i] = rand_round(rates[i]*size)
		s = np.sum(class_get)
		if s > size:
			class_get[np.argmax(class_get)] -= s-size
		resultvec = np.zeros((3,size), dtype=np.intp)
		got = 0
		for i in range(self.N_out):
			resultvec[:,got:got+class_get[i]] = self.get_testsamps_of_class(i,class_get[i])
			got += class_get[i]
		resultvec[:,got:] = self.get_random_points(size-got)	
		return self.make_stiches(resultvec)

In [26]:
from __future__ import division
import tensorflow as tf
sess = tf.InteractiveSession()

def Jacc(activations, truth, alpha=1e-4, beta=2.):
    """
    computes the Jaccard index
    activations: activation layer of the network (tensor)
    truth: ground truth. Must have same shape as activations
    """
    TPc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum(activations*truth,axis=0),axis=0),axis=0)
    FPc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum(activations*(1-truth),axis = 0),axis=0),axis=0)
    FNc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum((1-activations)*truth,axis = 0),axis=0),axis=0)
    return -tf.reduce_mean((TPc + alpha) / (TPc + FPc + FNc + beta))


def logJacc(activations, truth, alpha=1e-4, beta=2.):
    """
    computes the Jaccard index
    activations: activation layer of the network (tensor)
    truth: ground truth. Must have same shape as activations
    """
    TPc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum(activations*truth,axis=0),axis=0),axis=0)
    FPc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum(activations*(1-truth),axis = 0),axis=0),axis=0)
    FNc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum((1-activations)*truth,axis = 0),axis=0),axis=0)
    return -tf.reduce_mean(tf.log((TPc + alpha + 1e-4) / (TPc + FPc + FNc + beta + 2.)))

def individualJacc(activations, truth, alpha=1e-4, beta=2.):
    """
    computes the Jaccard index
    activations: activation layer of the network (tensor)
    truth: ground truth. Must have same shape as activations
    """
    TPc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum(activations*truth,axis=0),axis=0),axis=0)
    FPc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum(activations*(1-truth),axis = 0),axis=0),axis=0)
    FNc = tf.reduce_sum(tf.reduce_sum(tf.reduce_sum((1-activations)*truth,axis = 0),axis=0),axis=0)
    return (TPc + alpha + 1e-4) / (TPc + FPc + FNc + beta + 2.)

	
def cross_entropy(activations, truth):
    """
    computes the cross entropy
    activations: activation layer of the network (tensor)
    truth: ground truth. Must have same shape as activations
    """
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = truth, logits = activations))

def weight_variable(shape,name):
    initial = tf.truncated_normal(shape, stddev=0.6)
    return tf.Variable(initial,name=name)

def bias_variable(shape,name):
    initial = tf.constant(-0.1, shape=shape)
    return tf.Variable(initial,name=name)


class DSTLNet:

    def __init__(self, convSize = 17, channels = 16, hiddenUnits = 100, hiddenUnits2 = 60 features = 10, activations = 'sigmoid', 
                 loss = 'logJacc', learningRate = 0.005, minimizer = 'adam', loadFile = ''):
        """
        initialization of the net, setting all parameters. 
        convSize: size of the convolution patch (default:11)
        channels: number of channels in image (default:8)
        hiddenUnits: number of units in hidden layer (default:50)
        features: number of output features (default: 10)
        activations: type of activation function. Available functions: relu, sigmoid
        loss: type of loss function. Available loss functions: logJacc, jacc, crossEntropy
        loadFile: if not empty string, parameters will be loaded from specified file. Enter only filename without .ckpt as in DSTLNet.save(name) (default: empty string)
        learningRate: the learning Rate to be used during training (default: 0.1)
        optimizer: optimizer to be used during training (default: adam)
        """
        self.conv_size = convSize
        self.N_hidden = hiddenUnits
        self.features = features
        self.channels = channels
        if loss == 'logJacc':
            self.loss = logJacc
        elif loss == 'crossEntropy':
            self.loss == cross_entropy
        elif loss == 'jacc':
            self.loss = Jacc
        else:
            print('ERROR, loss function with name' + loss + ' not known. Please look at documentation')


        def conv2d(xt,Wt):
            """
            Wrapper for the tf conv2d function
            """
            return tf.nn.conv2d(xt,Wt,strides=[1,1,1,1], padding='VALID')
        
        def conv2d_SAME(xt,Wt):
            """
            Wrapper for the tf conv2d function
            """
            return tf.nn.conv2d(xt,Wt,strides=[1,1,1,1], padding='SAME')
        
        ac_func = []
        if activations == 'sigmoid':
            ac_func = tf.nn.sigmoid
        elif activations == 'relu':
            ac_func = tf.nn.relu
        else:
            print('ERROR, activation function with name' + activations + ' not known. Please look at documentation')


        self.truth = tf.placeholder(tf.float32, [None,None,None, features])
        self.x_in = tf.placeholder(tf.float32, [None, None,None,channels])
        #defining network
    
        '''
        self.W_conv1 = weight_variable([convSize,convSize,channels,self.N_hidden],name='W_conv1')
        self.b_conv1 = bias_variable([self.N_hidden],name='b_conv1')
        self.conv_1 = ac_func(conv2d(self.x_in, self.W_conv1)+self.b_conv1)

        self.W_conv2 = weight_variable([1,1,self.N_hidden,features], name='W_conv2')
        self.b_conv2 = bias_variable([features],name='b_conv2')
        self.out = tf.nn.sigmoid(conv2d(self.conv_1,self.W_conv2)+self.b_conv2)
        '''
        
        filtSize2 = 1
        
        self.W_conv1 = weight_variable([convSize,convSize,channels,hiddenUnits2],name='W_conv1')
        self.b_conv1 = bias_variable([hiddenUnits2],name='b_conv1')
        conv_1 = tf.nn.sigmoid(conv2d(self.x_in, self.W_conv1)+self.b_conv1)
        #conv_1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
        
        
        self.W_conv3 = weight_variable([filtSize2,filtSize2,hiddenUnits2,self.N_hidden],name='W_conv3')
        self.b_conv3 = bias_variable([self.N_hidden],name='b_conv3')
        conv_3 = tf.nn.sigmoid(conv2d_SAME(conv_1, self.W_conv3)+self.b_conv3)
        

        self.W_conv2 = weight_variable([1,1,self.N_hidden,features], name='W_conv2')
        self.b_conv2 = bias_variable([features],name='b_conv2')
        self.out = tf.nn.sigmoid(conv2d_SAME(conv_3,self.W_conv2)+self.b_conv2)
        
        
        if minimizer == 'adam':
            self.train_step = tf.train.AdamOptimizer(learningRate).minimize(self.loss(self.out, self.truth))
        elif minimizer == 'gradientDescent':
            self.train_step = tf.train.GradientDescentOptimizer(learningRate).minimize(self.loss(self.out, self.truth))
        else:
            print('ERROR: unknown minimizer')

        #initializing variables
        sess.run(tf.global_variables_initializer())
        self.saver = tf.train.Saver()
        if loadFile != '':
            self.saver.restore(sess, "/tmp/"+loadFile+'.ckpt')



    def train(self, dataWrap, miniBatchSize = 50):
        """
        trains the net
        datWrap: A DataWrapper object
        """
        batch = dataWrap.getMiniBatch(miniBatchSize)
        
    #       self.train_step = tf.train.AdamOptimizer(0.001).minimize(self.loss(self.out, self.truth))
    #       sess.run(tf.global_variables_initializer())
    #		test = dataWrap.getTestData(100)
    #		print('Jaccard loss:', self.eval_loss(test[0],test[1]))
        for i in range(int(50000/miniBatchSize)):
            #print(sess.run(self.W_conv1)[0,0,:,0])
            self.train_step.run(feed_dict={self.x_in: batch[0], self.truth: batch[1]})
            #print(sess.run(self.W_conv1)[0,0,:,0])
    #		print(sess.run(self.loss(self.out, self.truth), feed_dict={self.x_in: batch[0], self.truth: batch[1]}))
            batch = dataWrap.getMiniBatch(miniBatchSize)
    #		test = dataWrap.getTestData(100)
    #		print('Jaccard loss:', self.eval_loss(test[0],test[1]))

    def feedForward(self, data):
        """
        computes predictions for data
        data: image data of suitable shape
        """
        return sess.run(self.out, feed_dict={self.x_in:data})

    def eval_loss(self, data, truth, loss='iJacc'):
        if loss == 'logJacc':
            self.ev_loss = logJacc
        elif loss == 'crossEntropy':
            self.ev_loss == cross_entropy
        elif loss == 'jacc':
            self.ev_loss = Jacc
        elif loss == 'iJacc':
            self.ev_loss = individualJacc
        elif self.ev_loss == None:
            print('ERROR, loss function with name' + loss + ' not known. Please look at documentation')
        return sess.run(self.ev_loss(self.out, self.truth), feed_dict={self.x_in : data, self.truth : truth})




    def save(self, name):
        """
        saves model in /tmp/
        name: name of the savefile without .ckpt
        """
        save_path = self.saver.save(sess, "/tmp/"+name+".ckpt")
        print("Model saved in file: %s" % save_path)

In [ ]:
#import net
#import DataWrapper2 as dw
import numpy as np
import time

dat = DataWrapper(stichSize=15)
a,b = dat.getMiniBatch(100)

mynet = DSTLNet(activations = 'sigmoid', minimizer = 'adam', learningRate = 0.0001, hiddenUnits = 120, convSize = 15)

start = time.time()
for i in range(200):
    mynet.train(dat, miniBatchSize = 100)
    if i%5 == 0:
        print "step: ", i
        print('uniformness:',dat.uniformness)
        test = dat.getTestData(size=200)
        jl = mynet.eval_loss(test[0],test[1])
        print('Jaccard loss:', jl,  np.sum(jl)/10.)
        dat.uniformness *= 0.97
        print (time.time()- start)/60
        start = time.time()
        
    #if i > 500:
    #    dat.uniformness *= 0.95
    

mynet.save('mynet')


loading files...
... files loaded. Analyzing data...


In [12]:
start = time.time()
for i in range(1):
    mynet.train(dat, miniBatchSize = 100)
    if i%5 == 0:
        print "step: ", i
        print('uniformness:',dat.uniformness)
        test = dat.getTestData(size=500)
        jl = mynet.eval_loss(test[0],test[1])
        print('Jaccard loss:', jl,  np.sum(jl)/10.)
        dat.uniformness *= 0.97
        print (time.time()- start)/60
        start = time.time()
        
    #if i > 500:
    #    dat.uniformness *= 0.95
    

mynet.save('mynet')


step:  0
('uniformness:', 0.6689035031101793)
('Jaccard loss:', array([ 0.33857068,  0.24641646,  0.59076297,  0.22712862,  0.45981938,
        0.76626319,  0.65733105,  0.61059028,  0.42008361,  0.22593838], dtype=float32), 0.45429043769836425)
2.9012057662
Model saved in file: /tmp/mynet.ckpt


In [19]:

start = time.time()
for i in range(100):
    mynet.train(dat, miniBatchSize = 100)
    dat.uniformness *= 0.96

    print "step: ", i
    print('uniformness:',dat.uniformness)
    if i%5 == 0:
        
        test = dat.getTestData(size=500)
        jl = mynet.eval_loss(test[0],test[1])
        print('Jaccard loss:', jl,  np.sum(jl)/10.)
        dat.uniformness *= 0.92
    print (time.time()- start)/60
    start = time.time()


step:  0
('uniformness:', 0.5981544892949797)
('Jaccard loss:', array([ 0.36082187,  0.17760353,  0.54975832,  0.18423638,  0.5465458 ,
        0.73744404,  0.68286264,  0.60582268,  0.39749211,  0.22865835], dtype=float32), 0.44712457656860349)
2.90544921557
step:  1
('uniformness:', 0.5282900449453262)
2.92024614811
step:  2
('uniformness:', 0.5071584431475131)
3.03075243235
step:  3
('uniformness:', 0.4868721054216126)
2.8851540486
step:  4
('uniformness:', 0.4673972212047481)
2.86154824893
step:  5
('uniformness:', 0.4487013323565581)
('Jaccard loss:', array([ 0.4005169 ,  0.17552578,  0.56981379,  0.2068374 ,  0.43521136,
        0.70975876,  0.62838936,  0.58134389,  0.25242656,  0.20002258], dtype=float32), 0.4159846305847168)
2.8587104996
step:  6
('uniformness:', 0.3962930167373121)
2.85580553214
step:  7
('uniformness:', 0.3804412960678196)
2.86528085073
step:  8
('uniformness:', 0.36522364422510684)
2.90155651967
step:  9
('uniformness:', 0.35061469845610255)
2.8629458348
st

KeyboardInterrupt: 

In [ ]:
print maximu, minimu